# FordA Time‑Series Classification with 1‑D CNN
This notebook demonstrates how to classify the FordA engine‑sound dataset from the UCR/UEA archive using a simple 1‑D Convolutional Neural Network (CNN) implemented with **TensorFlow 2.x** and **Keras**.

## 1. Imports

In [24]:

import tensorflow as tf
import tensorflow_datasets as tfds, pkg_resources
import numpy as np, pathlib, zipfile, urllib
from urllib.error import HTTPError, URLError

print("tfds  :", tfds.__version__)
print("nightly installed?:", 'tfds-nightly' in {p.key for p in pkg_resources.working_set})

tfds  : 4.9.8+nightly
nightly installed?: True


## 2. Load the FordA dataset

In [ ]:
# ========== Dataset Loading with on-demand download ==========

def manual_load_ford_a():
    mirrors  = ["https://www.timeseriesclassification.com/aeon-toolkit/FordA.zip"]
    data_dir = pathlib.Path("data/forda_raw")         # 기준 폴더
    data_dir.mkdir(exist_ok=True)

    # Check file existence
    has_train = bool(list(data_dir.glob("**/FordA_TRAIN.ts")))
    has_test  = bool(list(data_dir.glob("**/FordA_TEST.ts")))
    need_dl   = not (has_train and has_test)

    if need_dl:
        zip_path = data_dir / "FordA.zip"
        for url in mirrors:
            try:
                print(f"Downloading {url}")
                req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
                with urllib.request.urlopen(req) as resp, open(zip_path, "wb") as f:
                    f.write(resp.read())
                print("     ↳ done")
                break
            except (HTTPError, URLError) as e:
                print("     ↳ failed:", e)
        else:
            raise RuntimeError("All mirrors failed.")

        with zipfile.ZipFile(zip_path, "r") as zf:
            zf.extractall(data_dir)
        zip_path.unlink()               # ZIP 삭제

    # .ts file reader 
    def _load(split):                   # split = "TRAIN" / "TEST"
        path = list(data_dir.glob(f"**/FordA_{split}.ts"))[0]
        print("Reading", path)
        series, labels = [], []
        with open(path) as f:
            for line in f:
                line = line.strip()
                if not line or line[0] in ("@", "#"):
                    continue
                ts, lab = line.split(":")
                series.append([float(v) for v in ts.split(",")])
                labels.append(int(lab))
        X = np.array(series, dtype=np.float32)
        y = ((np.array(labels) + 1) // 2).astype(np.float32)  # {-1,1}→{0,1}
        return X, y

    x_tr, y_tr = _load("TRAIN")
    x_te, y_te = _load("TEST")
    return x_tr, y_tr, x_te, y_te


# Load the dataset
x_train, y_train, x_test, y_test = manual_load_ford_a()
print("Shapes:", x_train.shape, y_train.shape)


Reading forda_raw/FordA_TRAIN.ts
Reading forda_raw/FordA_TEST.ts
Shapes: (3601, 500) (3601,)


## 2.1 FordA dataset 

## 3. Normalise signals and expand channel dimension

In [26]:

mean, std = x_train.mean(), x_train.std()

x_train = ((x_train - mean) / std)[..., np.newaxis]  # (N, 500, 1)
x_test  = ((x_test  - mean) / std)[..., np.newaxis]

# Map labels {-1, 1} to {0, 1}
y_train = ((y_train + 1) // 2).astype('float32')
y_test  = ((y_test  + 1) // 2).astype('float32')


## 4. Build the CNN model

In [27]:

model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(64, 5, activation='relu', input_shape=(500, 1)),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()


/usr/local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (None, 496, 64)        │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 248, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 244, 128)       │        41,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 244, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_1      │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 49,793 (194.50 KB)

 Trainable params: 49,793 (194.50 KB)

 Non-trainable params: 0 (0.00 B)

## 5. Train

In [28]:

history = model.fit(
    x_train, y_train,
    epochs=15,
    batch_size=128,
    validation_split=0.2
)


Epoch 1/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.5157 - loss: 0.6910 - val_accuracy: 0.6630 - val_loss: 0.6840
Epoch 2/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6617 - loss: 0.6757 - val_accuracy: 0.5201 - val_loss: 0.6525
Epoch 3/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5804 - loss: 0.6384 - val_accuracy: 0.6796 - val_loss: 0.6209
Epoch 4/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6981 - loss: 0.5993 - val_accuracy: 0.6921 - val_loss: 0.5887
Epoch 5/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.7184 - loss: 0.5508 - val_accuracy: 0.7198 - val_loss: 0.5339
Epoch 6/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.7343 - loss: 0.5121 - val_accuracy: 0.7448 - val_loss: 0.4962
Epoch 7/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.7652 - loss: 0.4723 - val_accuracy: 0.7545 - val_loss: 0.4740
Epoch 8/15
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.7761 - loss: 0.4455 - val_accuracy: 0.7226 - v

## 6. Evaluate

In [29]:

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"Test accuracy: {test_acc:.3f}")


Test accuracy: 0.836


## 7. Model Save & Load Model

In [ ]:
model_dir = "/model_/ford_a_model.h5"
model.save(model_dir)
# Load the model
loaded_model = tf.keras.models.load_model(model_dir)
# Check the loaded model
loaded_model.summary()